In [138]:
!pip install unidecode -q
!pip install contractions
!pip install nltk
!pip install gensim

In [41]:
import pandas as pd
import re

from nltk.corpus import stopwords
import contractions
from unidecode import unidecode
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
df_ = pd.read_csv('musk.csv', index_col=0)

df_

,type,author,text,created
0,Comment,UNCwesRPh,I had been on twitter prior to the musk takeov...,2023-01-30 18:49:31+00:00
2,Comment,wgp3,That article does not say what you imply at al...,2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,The OG musk duck lives on my wall.,2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,How dare he speak that way to the great and po...,2023-01-30 18:48:26+00:00
5,Comment,Copykill,Can’t wait to finally have an excuse not to sh...,2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"Like brain dead piranhas lmao, anything barely...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,What am I lying about? What's my agenda? Pleas...,2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,Hard disagree. I don't think he's a parody of ...,2023-05-01 07:07:36+00:00
85224,Comment,Viperions,Yeah. I think too many things are lining up ri...,2023-05-01 07:07:15+00:00


In [42]:
df_['type'].unique()

array(['Comment', '2023-04-30 15:47:51+00:00'], dtype=object)

In [43]:
# Remove column with only one value
df_.drop(columns=['type'])

,author,text,created
0,UNCwesRPh,I had been on twitter prior to the musk takeov...,2023-01-30 18:49:31+00:00
2,wgp3,That article does not say what you imply at al...,2023-01-30 18:49:28+00:00
3,HighAndDrunk,The OG musk duck lives on my wall.,2023-01-30 18:48:43+00:00
4,Louismaxwell23,How dare he speak that way to the great and po...,2023-01-30 18:48:26+00:00
5,Copykill,Can’t wait to finally have an excuse not to sh...,2023-01-30 18:48:10+00:00
...,...,...,...
85215,Emble12,"Like brain dead piranhas lmao, anything barely...",2023-05-01 07:09:53+00:00
85221,Pizza_in_Space,What am I lying about? What's my agenda? Pleas...,2023-05-01 07:08:43+00:00
85223,Da1realBigA,Hard disagree. I don't think he's a parody of ...,2023-05-01 07:07:36+00:00
85224,Viperions,Yeah. I think too many things are lining up ri...,2023-05-01 07:07:15+00:00


In [44]:
def remove_duplicates_and_na(df:pd.DataFrame) -> pd.DataFrame:
    '''Remove duplicates and NA values'''
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    return df

In [45]:
def lower(text:str) -> str:
    '''Lower all characters'''
    return text.lower()

In [46]:
def replace_contractions(text:str) -> str:
    '''Replace contractions in string of text'''
    return contractions.fix(text)

In [47]:
def remove_url(text:str) -> str:
   '''Remove URLs'''
   return re.sub(r"((www.[^\s]+)|(https?://[^\s]+))", " ", text)

In [48]:
def replace_to_unicode(text:str) -> str:
    '''Replace special characters to unicode standard'''
    return unidecode(text)

In [49]:
def text_to_word_list(text:str) -> str:
    ''' Pre process and convert texts to a list of words '''
    text = str(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"'s ", " is ", text)
    text = re.sub(r"'ve ", " have ", text)
    text = re.sub(r"n't ", " not ", text)
    text = re.sub(r"'re ", " are ", text)
    text = re.sub(r"'d ", " would ", text)
    text = re.sub(r"'ll ", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"/r/", " subreddit ", text)
    text = re.sub(r"0 0", "00", text)
    text = text.split()

    return text

In [50]:
df = df_.copy()
df = remove_duplicates_and_na(df)
df['text'] = df['text'].apply(lambda x: lower(x))
df['text'] = df['text'].apply(lambda x: remove_url(x))
df['text'] = df['text'].apply(lambda x: replace_contractions(x))
df['text'] = df['text'].apply(lambda x: replace_to_unicode(x))
df['text'] = df['text'].apply(lambda x: text_to_word_list(x))

In [51]:
sent = [row for row in df.text]
phrases = Phrases(sent, min_count=1, progress_per=50000)
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences[1]

['that',
 'article',
 'does_not',
 'say',
 'what',
 'you',
 'imply',
 'at',
 'all',
 'it',
 'only',
 'states',
 'that',
 'the',
 'feature',
 'turns',
 'off',
 'before',
 'crashes',
 'which',
 'tesla',
 'admits',
 'is',
 'default_behavior',
 'and',
 'has',
 'never',
 'tried',
 'to',
 'hide',
 'hence',
 'the',
 'whole',
 'counting',
 'as',
 'a',
 'crash',
 'if',
 'disengaged',
 'within',
 '5_seconds',
 'of',
 'a',
 'crash',
 'the',
 'article',
 'does',
 'say',
 'it',
 'brings',
 'into',
 'question',
 'the',
 'times',
 'musk',
 'said',
 'the',
 'feature',
 'was',
 'off',
 'it',
 'does_not',
 'give',
 'any_proof',
 'that',
 'the',
 'times',
 'musk',
 'said',
 'it',
 'was',
 'off',
 'were',
 'in',
 'fact',
 'referring_to',
 'crashes',
 'where',
 'it',
 'turned_off',
 'right',
 'before',
 'the',
 'crash',
 'it',
 'is',
 'speculation',
 'at',
 'best']

In [52]:
df['text'] = sentences

In [53]:
df

,type,author,text,created
0,Comment,UNCwesRPh,"[i, had, been, on, twitter, prior, to, the, mu...",2023-01-30 18:49:31+00:00
2,Comment,wgp3,"[that, article, does_not, say, what, you, impl...",2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,"[the, og, musk, duck, lives, on, my, wall]",2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,"[how_dare, he, speak, that, way, to, the, grea...",2023-01-30 18:48:26+00:00
5,Comment,Copykill,"[cannot_wait, to, finally, have, an_excuse, no...",2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"[like, brain_dead, piranhas, lmao, anything, b...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,"[what, am, i, lying_about, what, is, my, agend...",2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,"[hard, disagree, i, do_not, think, he, is, a, ...",2023-05-01 07:07:36+00:00
85224,Comment,Viperions,"[yeah, i, think, too_many, things, are, lining...",2023-05-01 07:07:15+00:00


In [54]:
def remove_stop_words(df:pd.DataFrame) -> pd.DataFrame:
    '''Remove stop words'''
    stops = set(stopwords.words('english'))
    df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stops])
    return df

In [55]:

df = remove_stop_words(df)
df = df[df.text.str.len() > 1]

In [56]:
df.shape

(61049, 4)

In [57]:
df.to_csv('musk_clean.csv')

In [58]:
df

,type,author,text,created
0,Comment,UNCwesRPh,"[twitter, prior, musk, takeover, i_am, talking...",2023-01-30 18:49:31+00:00
2,Comment,wgp3,"[article, does_not, say, imply, states, featur...",2023-01-30 18:49:28+00:00
3,Comment,HighAndDrunk,"[og, musk, duck, lives, wall]",2023-01-30 18:48:43+00:00
4,Comment,Louismaxwell23,"[how_dare, speak, way, great, powerful, musk, ...",2023-01-30 18:48:26+00:00
5,Comment,Copykill,"[cannot_wait, finally, an_excuse, shower, douc...",2023-01-30 18:48:10+00:00
...,...,...,...,...
85215,Comment,Emble12,"[like, brain_dead, piranhas, lmao, anything, b...",2023-05-01 07:09:53+00:00
85221,Comment,Pizza_in_Space,"[lying_about, agenda, please, correct, errors,...",2023-05-01 07:08:43+00:00
85223,Comment,Da1realBigA,"[hard, disagree, do_not, think, parody, elon_m...",2023-05-01 07:07:36+00:00
85224,Comment,Viperions,"[yeah, think, too_many, things, lining_up, rig...",2023-05-01 07:07:15+00:00
